In [ ]:
# Importieren der erforderlichen Bibliotheken
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter

In [ ]:
# Spark-Session erstellen
spark = SparkSession.builder \
    .appName("Yelp Visualisierung") \
    .config("spark.driver.memory", "8g") \
    .getOrCreate()

In [ ]:
# JSON-Daten mit PySpark laden
reviews_df = spark.read.json("yelp_academic_dataset_review.json")
users_df = spark.read.json("yelp_academic_dataset_user.json")
business_df = spark.read.json("yelp_academic_dataset_business.json")

In [ ]:
# Erste Untersuchung der Daten
reviews_df.printSchema()
users_df.printSchema()
business_df.printSchema()

In [ ]:
# PySpark-Operationen: Mergen der Daten
reviews = reviews_df.join(business_df, on="business_id", how="inner")

In [ ]:
# Spalten umbenennen, um Mehrdeutigkeiten zu vermeiden
reviews_df = reviews_df.withColumnRenamed("stars", "review_stars")
business_df = business_df.withColumnRenamed("stars", "business_stars")

# PySpark-Operationen: Mergen der Daten
reviews = reviews_df.join(business_df, on="business_id", how="inner")

# Konvertiere das PySpark DataFrame in Pandas DataFrame
reviews = reviews.select("user_id", "business_id", "review_stars", "name", "state").toPandas()
business_df = business_df.select("business_id", "name", "state").toPandas()

In [ ]:
# Anzahl Reviews insgesamt
total_reviews = reviews.shape[0]
print(f"Anzahl Reviews insgesamt: {total_reviews}")

# Anzahl verschiedener Nutzer
unique_users = reviews['user_id'].nunique()
print(f"Anzahl verschiedener Nutzer: {unique_users}")

# Anzahl verschiedener Unternehmen
unique_businesses = reviews['business_id'].nunique()
print(f"Anzahl verschiedener Businesses: {unique_businesses}")

# Anzahl verschiedener Bundesstaaten
all_states = business_df['state'].dropna().unique()
unique_states = len(all_states)
print(f"Anzahl verschiedener States: {unique_states}")

In [ ]:
# Doppelte Kombinationen von user_id und business_id finden
duplicates = reviews.groupby(['user_id', 'business_id']).size()
duplicate_count = (duplicates > 1).sum()
print(f"Anzahl der doppelten Bewertungen: {duplicate_count}")

In [ ]:
# Bewertungen pro Nutzer berechnen
user_reviews_count = reviews.groupby('user_id').size()

# Nutzer mit mindestens 1, 3 und 5 Bewertungen zählen
users_1 = (user_reviews_count == 1).sum()
users_min_1 = (user_reviews_count >= 1).sum()
users_min_30 = (user_reviews_count >= 30).sum()
users_min_50 = (user_reviews_count >= 50).sum()

print(f"Anzahl der Nutzer mit genau 1 Bewertung: {users_1}")
print(f"Anzahl der Nutzer mit mindestens 1 Bewertung: {users_min_1}")
print(f"Anzahl der Nutzer mit mindestens 30 Bewertungen: {users_min_30}")
print(f"Anzahl der Nutzer mit mindestens 50 Bewertungen: {users_min_50}")

In [ ]:
# Nutzer mit ausschließlich demselben Rating
user_unique_reviews = reviews.groupby('user_id')['review_stars'].nunique()
num_users_with_same_reviews = (user_unique_reviews == 1).sum()
print(f"Anzahl der Benutzer mit ausschließlich demselben Rating: {num_users_with_same_reviews}")

In [ ]:
# Bewertungen pro Business berechnen
business_reviews_count = reviews.groupby('business_id').size()

# Businesses mit mindestens 1, 3 und 5 Bewertungen zählen
businesses_min_1 = (business_reviews_count >= 1).sum()
businesses_min_3 = (business_reviews_count >= 3).sum()
businesses_min_5 = (business_reviews_count >= 5).sum()

print(f"Anzahl der Businesses mit mindestens 1 Bewertung: {businesses_min_1}")
print(f"Anzahl der Businesses mit mindestens 3 Bewertungen: {businesses_min_3}")
print(f"Anzahl der Businesses mit mindestens 5 Bewertungen: {businesses_min_5}")

In [ ]:
# Durchschnittliche Bewertung pro Business
avg_rating_per_business = reviews.groupby('business_id')['review_stars'].mean()
print(avg_rating_per_business.describe())

In [ ]:
# Durchschnittliche Bewertung pro Nutzer
avg_rating_per_user = reviews.groupby('user_id')['review_stars'].mean()
print(avg_rating_per_user.describe())

In [ ]:
# Ratings-Verteilung
rating_distribution = reviews['review_stars'].value_counts().sort_index()

# Visualisierungen
plt.figure(figsize=(10, 6))
rating_distribution.plot(kind='bar', color='skyblue')
plt.xlabel('Rating')
plt.ylabel('Anzahl der Bewertungen')

# Ticks für die Y-Achse festlegen (lineare Skala)
ticks = [1_000_000, 2_000_000, 3_000_000]
plt.yticks(ticks)  # Manuelle Ticks setzen

# Y-Achse formatieren: Keine wissenschaftliche Notation, sondern Klartext
def linear_format(x, _):
    return f'{int(x):,}'

plt.gca().yaxis.set_major_formatter(FuncFormatter(linear_format))

# Achsenbeschriftungen anpassen
plt.xticks(rotation=0)  # X-Achse bleibt aufrecht

# Diagramm speichern
plt.savefig("RatingsVerteilung_Yelp.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Verteilung der Bewertungen pro Nutzer
user_reviews_count.plot.hist(bins=50, color='lightgreen')
plt.xlabel('Anzahl der Bewertungen')
plt.ylabel('Anzahl der Nutzer')
plt.yscale('log')

# Y-Achse mit normalen Zahlen beschriften
def log_format(x, _):
    return f'{int(x):,}' if x >= 1 else ''

plt.gca().yaxis.set_major_formatter(FuncFormatter(log_format))
plt.savefig("AnzahlBewertungenNutzer_Yelp.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Top 10 Nutzer mit den meisten Bewertungen
top_users = user_reviews_count.nlargest(10)
top_users.plot(kind='bar', color='orange', title='Top 10 Nutzer mit den meisten Bewertungen')
plt.xlabel('Nutzer-ID')
plt.ylabel('Anzahl der Bewertungen')
plt.show()

In [ ]:
# Histogramm der durchschnittlichen Bewertungen pro Benutzer
plt.figure(figsize=(10, 6))
sns.histplot(avg_rating_per_user, bins=20, color='purple')
plt.title('Durchschnittliche Bewertung pro Benutzer')
plt.xlabel('Durchschnittliche Bewertung')
plt.ylabel('Anzahl der Benutzer')
plt.show()

In [ ]:
# Verteilung der Anzahl von Bewertungen pro Unternehmen
business_reviews_count = reviews.groupby('business_id').size()

plt.figure(figsize=(10, 6))
plt.hist(business_reviews_count, bins=50, color='lightgreen')
plt.title('Verteilung der Anzahl von Bewertungen pro Business')
plt.xlabel('Anzahl der Bewertungen')
plt.ylabel('Anzahl der Businesses')

# Achse auf logarithmische Skala setzen
plt.yscale('log')

# Y-Achse mit normalen Zahlen beschriften
def log_format(x, _):
    return f'{int(x):,}' if x >= 1 else ''

plt.gca().yaxis.set_major_formatter(FuncFormatter(log_format))

plt.show()

In [ ]:
# Durchschnittliche Bewertung der Top 10 Unternehmen (nach Anzahl Bewertungen)
top_business_counts = reviews['business_id'].value_counts().head(10)
top_business_titles = business_df[business_df['business_id'].isin(top_business_counts.index)]
business_id_to_name = dict(zip(top_business_titles['business_id'], top_business_titles['name']))
top_avg_ratings = avg_rating_per_business.loc[top_business_counts.index]
top_avg_ratings.index = top_avg_ratings.index.map(business_id_to_name)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_avg_ratings.index, y=top_avg_ratings.values, palette='viridis')
plt.title('Durchschnittliche Bewertung der Top 10 Businesses')
plt.xlabel('Business-Name')
plt.ylabel('Durchschnittliche Bewertung')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Top 10 Unternehmen mit den meisten Bewertungen
top_business_counts = reviews['business_id'].value_counts().head(10)
top_business_titles = business_df[business_df['business_id'].isin(top_business_counts.index)]
business_id_to_name = dict(zip(top_business_titles['business_id'], top_business_titles['name']))
top_business_counts.index = top_business_counts.index.map(business_id_to_name)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_business_counts.index, y=top_business_counts.values, palette='viridis')
plt.title('Top 10 Businesses mit den meisten Bewertungen')
plt.xlabel('Business-Name')
plt.ylabel('Anzahl der Bewertungen')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Beende die Spark-Session
spark.stop()